<a href="https://colab.research.google.com/github/jsebastianber12/hola123/blob/main/Copia_de_Untitled16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montar Google Drive y cargar datos
from google.colab import drive
drive.mount('/content/gdrive')

# Librerías necesarias
import numpy as np
import h5py
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Concatenate, BatchNormalization, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# Cargar datos desde Google Drive
data_path = "/content/gdrive/MyDrive/Project_2024-2"

# Cargar datos de entrenamiento
dataset = h5py.File(data_path + '/training_set_500.h5', 'r')
X_train = np.array(dataset.get('data'))
y_train = np.array(dataset.get('labels'))
dataset.close()

# Normalización de datos
X_train = X_train / np.max(X_train)

# Convertir etiquetas a formato one-hot
nclases = 3
y_train = to_categorical(y_train, nclases)

# Cargar datos de prueba
dataset = h5py.File(data_path + '/test_without_labels_200.h5', 'r')
X_test = np.array(dataset.get('data'))
dataset.close()

# Normalización de datos de prueba
X_test = X_test / np.max(X_test)

# Función para crear ventanas deslizantes
def create_sliding_windows(data, window_size, stride):
    windows = []
    for signal in data:
        signal_windows = [
            signal[i:i+window_size]
            for i in range(0, len(signal) - window_size + 1, stride)
        ]
        windows.append(signal_windows)
    return np.array(windows)

# Parámetros para ventanas deslizantes
window_size = 256
stride = 128

# Crear ventanas deslizantes para entrenamiento y prueba
X_train_windows = create_sliding_windows(X_train.squeeze(-1), window_size, stride).reshape(-1, window_size, 1)
X_test_windows = create_sliding_windows(X_test.squeeze(-1), window_size, stride).reshape(-1, window_size, 1)

# Ajustar etiquetas para ventanas
y_train_windows = np.repeat(y_train, X_train_windows.shape[0] // X_train.shape[0], axis=0)

# Función para calcular intervalos RR
from scipy.signal import find_peaks

def calculate_rr_intervals(signals, fs=250):
    rr_intervals = []
    for signal in signals:
        peaks, _ = find_peaks(signal.flatten(), height=0.5, distance=fs*0.6)
        rr = np.diff(peaks) / fs
        rr_intervals.append(rr[:10] if len(rr) >= 10 else np.pad(rr, (0, 10 - len(rr)), constant_values=0))
    return np.array(rr_intervals)

X_train_rr = calculate_rr_intervals(X_train.squeeze(-1))
X_test_rr = calculate_rr_intervals(X_test.squeeze(-1))

# Normalizar intervalos RR
X_train_rr = X_train_rr / np.max(X_train_rr)
X_test_rr = X_test_rr / np.max(X_test_rr)

# Repetir intervalos RR para que coincidan con las ventanas deslizantes
X_train_rr_repeated = np.repeat(X_train_rr, X_train_windows.shape[0] // X_train_rr.shape[0], axis=0)
X_test_rr_repeated = np.repeat(X_test_rr, X_test_windows.shape[0] // X_test_rr.shape[0], axis=0)

# Dividir datos en entrenamiento y validación
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_windows, y_train_windows, test_size=0.2, random_state=42
)
X_train_rr_split, X_val_rr_split = train_test_split(X_train_rr_repeated, test_size=0.2, random_state=42)

# Función para crear el modelo CNN + LSTM optimizado
def create_cnn_lstm_model_optimized(window_size, rr_size):
    ecg_input = Input(shape=(window_size, 1), name='ECG_Input')

    # Bloque CNN ajustado
    x = Conv1D(16, kernel_size=3, kernel_initializer='he_normal')(ecg_input)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)

    x = Conv1D(32, kernel_size=3, kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)

    rr_input = Input(shape=(rr_size,), name='RR_Input')
    y = Dense(32, kernel_initializer='he_normal')(rr_input)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.3)(y)

    combined = Concatenate()([x, y])

    z = Dense(64, kernel_initializer='he_normal')(combined)
    z = LeakyReLU(alpha=0.1)(z)
    z = Dropout(0.4)(z)

    z = Dense(32, kernel_initializer='he_normal')(z)
    z = LeakyReLU(alpha=0.1)(z)
    z = Dropout(0.4)(z)

    output = Dense(nclases, activation='softmax', name='Output')(z)

    model = Model(inputs=[ecg_input, rr_input], outputs=output)
    return model

# Crear modelo optimizado
model = create_cnn_lstm_model_optimized(window_size=256, rr_size=10)

# Verificar número de parámetros entrenables
model.summary()

# Compilar modelo con Nadam
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)

# Entrenar el modelo
history = model.fit(
    [X_train_split, X_train_rr_split],
    y_train_split,
    validation_data=([X_val_split, X_val_rr_split], y_val_split),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Realizar predicciones finales
y_pred = model.predict([X_test_windows, X_test_rr_repeated])

# Promediar predicciones para reducir al tamaño original
n_windows_per_signal = X_test_windows.shape[0] // X_test.shape[0]
y_pred_reshaped = y_pred.reshape(X_test.shape[0], n_windows_per_signal, -1)
y_pred_avg = np.mean(y_pred_reshaped, axis=1)

# Convertir a formato One-Hot Encoding
y_pred_classes = np.argmax(y_pred_avg, axis=1)
y_pred_one_hot = to_categorical(y_pred_classes, num_classes=nclases)

# Calcular F1-Score en datos de validación
y_val_pred = model.predict([X_val_split, X_val_rr_split])
y_val_pred_classes = np.argmax(y_val_pred, axis=1)
y_val_true_classes = np.argmax(y_val_split, axis=1)

f1 = f1_score(y_val_true_classes, y_val_pred_classes, average='weighted')
print(f"F1-Score en validación: {f1:.4f}")

# Guardar modelo y predicciones
model.save('/content/gdrive/MyDrive/GROUP_C1_2171513_2172323_Carlos_Bautista_Sebastian_Bernal_model.h5')
np.save('/content/gdrive/MyDrive/GROUP_C1_2171513_2172323_Carlos_Bautista_Sebastian_Bernal_prediction_fixed', y_pred_one_hot)

print("\nModelo y predicciones guardados exitosamente.")